In [1]:
import polars as pl

In [9]:
data1 = pl.read_excel(source='df/data_1.xlsx', sheet_name="Результаты поиска")

In [10]:
data1.head(1)

Уровень,Заказчик,ИНН заказчика,Стоимость (руб.) Заказчик,Реестровый номер публикации,Идентификационный код закупки,Сфера деятельности,Наименование публикации,Регион поставки,Город поставки,Дата публикации,Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ,Дата начала подачи заявок/Дата начала исполнения контракта / Дата публикации ППГ,Дата окончания проведения торгов,Поставщик,ИНН поставщика,Победитель,Статус допуска,Стоимость (руб.) Поставщик,"Снижение на торгах,%",Форма публикации,Тип торгов,Торговая площадка,Электронные торги,Обеспечение заявки (руб.),"Обеспечение заявки, %",Обеспечение контракта (руб.),"Обеспечение контракта, %",Банковское \ казначейское сопровождение,Источник финансирования,Ссылка на источник
i64,str,i64,f64,str,null,str,str,str,str,datetime[ms],datetime[ms],datetime[ms],datetime[ms],str,i64,null,null,f64,null,str,str,str,str,f64,null,null,null,null,str,str
1,"""АО ""СОЦСФЕРА""""",5528022883,1.14e6,"""SBR003-200007203100001""",null,"""Неизвестно""","""Транспортные услуги""","""Омская область""","""Омская область""",2020-01-13 09:23:00,2020-01-31 23:00:00,2020-01-14 00:00:00,2020-02-07 08:00:00,"""""",null,null,null,null,null,"""Торговая процедура""","""Запрос предложений в электронн…","""Сбербанк-АСТ""","""Неизвестно""",0.0,null,null,null,null,"""Неизвестно""","""СБЕРБАНК-АСТ 223"""


In [12]:
data6.shape

(763198, 31)

In [6]:
data2 = pl.read_excel(source='df/data_2.xlsx', sheet_name="Результаты поиска")
data3 = pl.read_excel(source='df/data_3.xlsx', sheet_name="Результаты поиска")
data4 = pl.read_excel(source='df/data_4.xlsx', sheet_name="Результаты поиска")
data5 = pl.read_excel(source='df/data_5.xlsx', sheet_name="Результаты поиска")

In [11]:
data6 = pl.read_excel(source='df/data_6.xlsx', sheet_name="Результаты поиска")
data7 = pl.read_excel(source='df/data_7.xlsx', sheet_name="Результаты поиска")
data8 = pl.read_excel(source='df/data_8.xlsx', sheet_name="Результаты поиска")
data9 = pl.read_excel(source='df/data_9.xlsx', sheet_name="Результаты поиска")
data10 = pl.read_excel(source='df/data_10.xlsx', sheet_name="Результаты поиска")
data11 = pl.read_excel(source='df/data_11.xlsx', sheet_name="Результаты поиска")
data12 = pl.read_excel(source='df/data_12.xlsx', sheet_name="Результаты поиска")
data13 = pl.read_excel(source='df/data_13.xlsx', sheet_name="Результаты поиска")
data14 = pl.read_excel(source='df/data_14.xlsx', sheet_name="Результаты поиска")
data15 = pl.read_excel(source='df/data_15.xlsx', sheet_name="Результаты поиска")

In [6]:
data1.head(1)

Уровень,Заказчик,ИНН заказчика,Стоимость (руб.) Заказчик,Реестровый номер публикации,Идентификационный код закупки,Сфера деятельности,Наименование публикации,Регион поставки,Город поставки,Дата публикации,Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ,Дата начала подачи заявок/Дата начала исполнения контракта / Дата публикации ППГ,Дата окончания проведения торгов,Поставщик,ИНН поставщика,Победитель,Статус допуска,Стоимость (руб.) Поставщик,"Снижение на торгах,%",Форма публикации,Тип торгов,Торговая площадка,Электронные торги,Обеспечение заявки (руб.),"Обеспечение заявки, %",Обеспечение контракта (руб.),"Обеспечение контракта, %",Банковское \ казначейское сопровождение,Источник финансирования,Ссылка на источник
i64,str,i64,f64,str,null,str,str,str,str,datetime[ms],datetime[ms],datetime[ms],datetime[ms],str,i64,null,null,f64,null,str,str,str,str,f64,null,null,null,null,str,str
1,"""АО ""СОЦСФЕРА""""",5528022883,1.14e6,"""SBR003-200007203100001""",null,"""Неизвестно""","""Транспортные услуги""","""Омская область""","""Омская область""",2020-01-13 09:23:00,2020-01-31 23:00:00,2020-01-14 00:00:00,2020-02-07 08:00:00,"""""",null,null,null,null,null,"""Торговая процедура""","""Запрос предложений в электронн…","""Сбербанк-АСТ""","""Неизвестно""",0.0,null,null,null,null,"""Неизвестно""","""СБЕРБАНК-АСТ 223"""


In [21]:
data2.select(pl.col('ИНН поставщика').unique())

ИНН поставщика
null
null


In [14]:
data = pl.concat([data1, data2])#, data3, data5, data6, data7, data8, data9, data10, data11, data12, data13, data14, data15])

SchemaError: type String is incompatible with expected type Null

Оставляем только те строки, где известен победитель и есть ИНН заказчика и поставщика.

In [471]:
fdata1 = data1.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata2 = data2.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata3 = data3.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata4 = data4.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata5 = data5.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())

In [476]:
data = pl.concat([fdata1, fdata2, fdata3, fdata5])
data = data.with_columns(pl.col("ИНН поставщика").str.to_integer())
data = pl.concat([data, fdata4])
data.shape

(168453, 18)

Убираем дупликаты

In [477]:
data = data.unique(maintain_order=True)
data.shape

(159192, 18)

In [478]:
data = data.rename({'Дата \nокончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ':'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                    'Дата начала подачи заявок/Дата начала исполнения контракта / Дата публикации ППГ' : 'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                    'Дата  окончания проведения торгов' : 'Дата окончания проведения торгов'})

Оставляем только те строки, где известно снижение на торгах

In [479]:
data = data.filter(pl.col("Снижение на торгах,%").is_not_null())
data.shape

(130616, 18)

Уберем нереалистичные значения в ставках в Снижение на торгах,% 

In [480]:
data = data.filter((pl.col('Снижение на торгах,%')>=-100))
data.shape

(130254, 18)

Теперь работаем с датами

In [481]:
ddata = data.with_columns(pl.col('Дата публикации').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата окончания проведения торгов').str.to_date(format="%m-%d-%y", strict=False))

In [482]:
data = ddata.filter(pl.col('Дата публикации').is_not_null() & 
                         pl.col('Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ').is_not_null() & 
                         pl.col('Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ').is_not_null() & 
                         pl.col('Дата окончания проведения торгов').is_not_null())
data.shape

(118302, 18)

Отбираем тендеры, где известен победитель

In [483]:
data = data.with_columns(pl.col('Стоимость \n(руб.) Заказчик').fill_null(0),
                         pl.col('Тип торгов').fill_null('n'))

In [484]:
data = data.with_columns(pl.when(pl.col('Победитель')==pl.lit('Победитель')).then(1).otherwise(0).alias('Есть победитель'))
data.shape

(118302, 19)

In [485]:
data_group = data.group_by(['Уровень',
                            'Заказчик',
                            'ИНН заказчика',
                            'Стоимость \n(руб.) Заказчик',
                            'Сфера деятельности',
                            'Регион поставки',
                            'Город поставки',
                            'Дата публикации',
                            'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                            'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                            'Дата окончания проведения торгов',
                            'Форма публикации',
                            'Тип торгов']).agg(pl.col('Есть победитель').sum().alias('Num winner'))
data_group.shape

(49825, 14)

In [486]:
data_new = data.join(data_group,
                     how='left',
                     on=['Уровень',
                        'Заказчик',
                        'ИНН заказчика',
                        'Стоимость \n(руб.) Заказчик',
                        'Сфера деятельности',
                        'Регион поставки',
                        'Город поставки',
                        'Дата публикации',
                        'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                        'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                        'Дата окончания проведения торгов',
                        'Форма публикации',
                        'Тип торгов'])
data_new.shape

(118302, 20)

In [487]:
data = data_new.filter(pl.col('Num winner')>=1)
data.shape

(17919, 20)

In [488]:
data = data.drop(["Есть победитель", "Num winner"])

Добавляем реестр недобросовестных поставщиков

In [489]:
notrust = pl.read_json('notrust_list/res.json')

In [490]:
notrust = notrust.rename({'suplier_name':'Поставщик', 'suplier_inn':'ИНН поставщика', 'no_trust_now':'РНП сейчас', 'no_trust_before':'РНП ранее'})
notrust = notrust.with_columns(pl.col('ИНН поставщика').str.to_integer())

In [491]:
data = data.join(other=notrust.select(['ИНН поставщика', 'РНП сейчас', 'РНП ранее']), on='ИНН поставщика', how='left')
data = data.with_columns(pl.col('РНП сейчас').fill_null(0), pl.col('РНП ранее').fill_null(0))

In [492]:
data.filter(pl.col('РНП ранее')==1).shape

(15, 20)

Записываем результат

In [493]:
data.write_excel( dtype_formats={pl.Date: "yyyy-mm-dd"})

In [494]:
data.shape

(17924, 20)

-------------

In [495]:
data2022 = pl.read_excel(source='data_2022.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")

ComputeError: could not parse `1934/ОКЭ-ДЖВ/22` as dtype `i64` at column 'Реестровый номер публикации' (column number 5)

The current offset in the file is 341113188 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `dtypes` argument
- setting `ignore_errors` to `True`,
- adding `1934/ОКЭ-ДЖВ/22` to the `null_values` list.

Original error: ```remaining bytes non-empty```

In [ ]:
data2022.shape